# MARATONA BEHIND THE CODE 2020

## DESAFIO 6 - LIT - Marcos Ferreira

<hr>

## Installing Libs

In [1]:
%%time
!pip install scikit-learn==0.22.0 --upgrade

Requirement already up-to-date: scikit-learn==0.22.0 in c:\users\marcospauloferreira\anaconda2\envs\py36\lib\site-packages (0.22)
Wall time: 5.26 s


## Imports

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, make_scorer, f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import StratifiedKFold

### Download training data (csv) to Pandas Dataframe

In [3]:
#!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/training_dataset.csv
#df_training_dataset = pd.read_csv(r'training_dataset.csv')
df_training_dataset = pd.read_csv(r'C:/Users/MARCOSPAULOFerreira/Anaconda2/MARATONA2020/desafio-6-2020/dataset/training_dataset.csv')
df_training_dataset.tail()

id     graduacao                universidade  \
15852  620397030.0   Bacharelado                         UFF   
15853  229931283.0   Bacharelado                         NaN   
15854  480674907.0     Tecnólogo                        UNIP   
15855  559626807.0  SEM FORMAÇÃO  UNIVERSIDADE NOVE DE JULHO   
15856  743652801.0           NaN                      FGV-RJ   

                     profissao  organizacao  pretende_fazer_cursos_lit  \
15852                   Outros     Borracha                        0.0   
15853                 Advogado          NaN                        0.0   
15854  Sócio/Dono/Proprietário          NaN                        0.0   
15855                 Advogado       Estado                        0.0   
15856                      NaN  Siderurgica                        1.0   

       interesse_mba_lit  importante_ter_certificado  horas_semanais_estudo  \
15852                0.0                         1.0                    8.0   
15853                0.0                         1.0                    7.0   
15854                NaN                         1.0                    7.0   
15855                0.0                         1.0                   10.0   
15856                1.0                         1.0                    9.0   

      como_conheceu_lit  total_modulos  modulos_iniciados  \
15852               NaN           10.0                NaN   
15853          Linkedin           42.0               17.0   
15854            Outros           30.0                9.0   
15855               NaN          226.0              102.0   
15856            Outros          125.0               98.0   

       modulos_finalizados  certificados categoria  
15852                  NaN           NaN   perfil6  
15853                 15.0           NaN   perfil5  
15854                  8.0           0.0   perfil5  
15855                 93.0           1.0   perfil1  
15856                 97.0           1.0   perfil1

In [4]:
# Exibindo os dados ausentes do conjunto de dados.
print("Valores nulos no df_training_dataset antes das transformaçoes: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset antes das transformaçoes: 

id                            1912
graduacao                     1907
universidade                  1937
profissao                     1880
organizacao                   1896
pretende_fazer_cursos_lit     1868
interesse_mba_lit             1854
importante_ter_certificado    1939
horas_semanais_estudo         1898
como_conheceu_lit             1942
total_modulos                 1870
modulos_iniciados             1813
modulos_finalizados           1933
certificados                  1878
categoria                        0
dtype: int64



## Tratando campos NaN no dataset de Trainamento.

In [5]:
#Nan to Zero nas colunas pretende_fazer_cursos_lit, interesse_mba_lit,
#importante_ter_certificado, modulos_finalizados, certificados
f = ['pretende_fazer_cursos_lit','interesse_mba_lit','importante_ter_certificado','modulos_finalizados','certificados']
values = {f[0]: 0, f[1]: 0, f[2]: 0, f[3]: 0, f[4]: 0}
df_training_dataset.fillna(value=values,inplace=True)

#Average: horas_semanais_estudo
df_training_dataset['horas_semanais_estudo'].fillna((df_training_dataset['horas_semanais_estudo'].mean()), inplace=True)

#modulos_iniciados Se for Nan transformar no mesmo valor de modulos_finalizados
df_training_dataset.modulos_iniciados.fillna(df_training_dataset.modulos_finalizados, inplace=True)

#total_modulos - Se for Nan transformar no mesmo valor de modulos_iniciados.
df_training_dataset.total_modulos.fillna(df_training_dataset.modulos_iniciados, inplace=True)

#Nan to 'Nao Informado' nas colunas graduacao, universidade, profissao, organizacao, como_conheceu_lit
f = ['graduacao','universidade','profissao','organizacao','como_conheceu_lit']
values = {f[0]: 'Nao Informado', f[1]: 'Nao Informado', f[2]: 'Nao Informado', f[3]: 'Nao Informado', f[4]: 'Nao Informado'}
df_training_dataset.fillna(value=values,inplace=True)

# Exibindo os dados ausentes do conjunto de dados apos fillna
print("Valores nulos no df_training_dataset apos transformaçoes: \n\n{}\n".format(df_training_dataset.isnull().sum(axis = 0)))

Valores nulos no df_training_dataset apos transformaçoes: 

id                            1912
graduacao                        0
universidade                     0
profissao                        0
organizacao                      0
pretende_fazer_cursos_lit        0
interesse_mba_lit                0
importante_ter_certificado       0
horas_semanais_estudo            0
como_conheceu_lit                0
total_modulos                    0
modulos_iniciados                0
modulos_finalizados              0
certificados                     0
categoria                        0
dtype: int64



### Codificando variáveis categóricas via Label Encoding

In [6]:
#Deep copy de df_training_dataset para df_train
df_train = df_training_dataset.copy(deep=True)

In [7]:
#Instanciar Encoders para cada campo.
le_graduacao = LabelEncoder()
le_universidade = LabelEncoder()
le_profissao = LabelEncoder()
le_organizacao = LabelEncoder()
le_como_conheceu_lit = LabelEncoder()

#Encode dos campos para code_<campo>
df_train["code_graduacao"] = le_graduacao.fit_transform(df_train["graduacao"])
df_train["code_universidade"] = le_graduacao.fit_transform(df_train["universidade"])
df_train["code_profissao"] = le_graduacao.fit_transform(df_train["profissao"])
df_train["code_organizacao"] = le_graduacao.fit_transform(df_train["organizacao"])
df_train["code_como_conheceu_lit"] = le_graduacao.fit_transform(df_train["como_conheceu_lit"])

df_train.head()

id       graduacao                           universidade  \
0  915132186.0  Especialização                                   UNIP   
1  183037660.0             MBA                          Nao Informado   
2  681614019.0     Bacharelado                                UNICAMP   
3  398921329.0    SEM FORMAÇÃO                                   UFRN   
4  522842284.0       Tecnólogo  INSPER INSTITUTO DE ENSINO E PESQUISA   

         profissao        organizacao  pretende_fazer_cursos_lit  \
0  Analista Senior   Eletrodomesticos                        1.0   
1       Supervisor         e-commerce                        0.0   
2           Outros         e-commerce                        1.0   
3           Outros  Eletroeletronicos                        0.0   
4  SEM EXPERIÊNCIA   Eletrodomesticos                        0.0   

   interesse_mba_lit  importante_ter_certificado  horas_semanais_estudo  \
0                1.0                         1.0                    6.0   
1                0.0                         1.0                    6.0   
2                1.0                         1.0                   10.0   
3                1.0                         1.0                    5.0   
4                0.0                         1.0                    7.0   

  como_conheceu_lit  total_modulos  modulos_iniciados  modulos_finalizados  \
0            Google           28.0                5.0                  4.0   
1     Nao Informado           66.0               36.0                 34.0   
2        Saint Paul           27.0               27.0                 27.0   
3         Instagram           29.0               10.0                  6.0   
4            Google           67.0               49.0                 44.0   

   certificados categoria  code_graduacao  code_universidade  code_profissao  \
0           0.0   perfil6               1                 14               2   
1           0.0   perfil2               3                  6              11   
2           0.0   perfil2               0                 13               9   
3           0.0   perfil5               5                 11               9   
4           0.0   perfil2               6                  5              10   

   code_organizacao  code_como_conheceu_lit  
0                 1                       1  
1                10                       6  
2                10                       8  
3                 2                       2  
4                 1                       1

## Treinando um classificador RANDOM FOREST CLASSIFIER
####    Otimizacao de Hyperparametros usando GridsearchCV em notebook pessoal.
##### {'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'n_estimators': 1500}
##### Acc: 0.8082864571465158

In [8]:
df_train.columns

Index(['id', 'graduacao', 'universidade', 'profissao', 'organizacao',
       'pretende_fazer_cursos_lit', 'interesse_mba_lit',
       'importante_ter_certificado', 'horas_semanais_estudo',
       'como_conheceu_lit', 'total_modulos', 'modulos_iniciados',
       'modulos_finalizados', 'certificados', 'categoria', 'code_graduacao',
       'code_universidade', 'code_profissao', 'code_organizacao',
       'code_como_conheceu_lit'],
      dtype='object')

In [9]:
features = [ 'pretende_fazer_cursos_lit', 'interesse_mba_lit',
             'importante_ter_certificado', 'horas_semanais_estudo',
             'total_modulos', 'modulos_iniciados', 'modulos_finalizados',
             'certificados', 'code_graduacao', 'code_universidade',
             'code_profissao', 'code_organizacao', 'code_como_conheceu_lit'
           ]
target = ['categoria']  ## NÃO TROQUE O NOME DA VARIÁVEL TARGET.

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_train[features]
y = df_train[target]

In [10]:
#Instanciar e treinar modelo com todos dados de treinamento.
RFC = RandomForestClassifier(criterion='gini', max_depth=10, max_features='auto', n_estimators=1500)
RFC.fit(X,y.values.ravel())

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Download da "folha de respostas" para predicao

In [11]:
#!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-3/master/to_be_scored.csv
#df_to_be_scored = pd.read_csv(r'to_be_scored.csv')
df_to_be_scored = pd.read_csv(r'C:/Users/MARCOSPAULOFerreira/Anaconda2/MARATONA2020/desafio-6-2020/dataset/to_be_scored.csv')
df_to_be_scored.tail()

id       graduacao                universidade    profissao  \
995  741745189    SEM FORMAÇÃO  UNIVERSIDADE NOVE DE JULHO  Coordenador   
996  996387882     Bacharelado                        UFRN     Assessor   
997  668869418     Bacharelado                      UFSCar      Gerente   
998  676531718       Tecnólogo                       FATEC       Outros   
999  388402340  Especialização                        UNIP     Advogado   

     organizacao  pretende_fazer_cursos_lit  interesse_mba_lit  \
995     Borracha                          0                  0   
996       Estado                          1                  1   
997  Siderurgica                          1                  1   
998     servicos                          1                  1   
999    Montadora                          0                  0   

     importante_ter_certificado  horas_semanais_estudo  \
995                           1                      2   
996                           1                      8   
997                           1                      9   
998                           1                      9   
999                           1                      9   

              como_conheceu_lit  total_modulos  modulos_iniciados  \
995                     YouTube            245                 63   
996                      Google            163                111   
997                    Facebook             56                 10   
998  Mídia (revista/jornal/web)            138                 60   
999  Mídia (revista/jornal/web)             22                 20   

     modulos_finalizados  certificados  
995                   45             1  
996                   96             0  
997                    4             0  
998                   30             0  
999                   19             0

In [12]:
# Exibindo os dados ausentes do conjunto para resposta
print("Valores nulos no df_to_be_scored antes das transformaçoes: \n\n{}\n".format(df_to_be_scored.isnull().sum(axis = 0)))

Valores nulos no df_to_be_scored antes das transformaçoes: 

id                            0
graduacao                     0
universidade                  0
profissao                     0
organizacao                   0
pretende_fazer_cursos_lit     0
interesse_mba_lit             0
importante_ter_certificado    0
horas_semanais_estudo         0
como_conheceu_lit             0
total_modulos                 0
modulos_iniciados             0
modulos_finalizados           0
certificados                  0
dtype: int64



#### Nenhum NaN nos dataframe to be scored.
#### Codificar as Colunas Categoricas com os Encoders criados acima com os dados de treino

In [13]:
#Deep copy de df_training_dataset para df_train
df_tbs = df_to_be_scored.copy(deep=True)
#Encode dos campos para code_<campo>
df_tbs["code_graduacao"] = le_graduacao.fit_transform(df_tbs["graduacao"])
df_tbs["code_universidade"] = le_graduacao.fit_transform(df_tbs["universidade"])
df_tbs["code_profissao"] = le_graduacao.fit_transform(df_tbs["profissao"])
df_tbs["code_organizacao"] = le_graduacao.fit_transform(df_tbs["organizacao"])
df_tbs["code_como_conheceu_lit"] = le_graduacao.fit_transform(df_tbs["como_conheceu_lit"])

In [14]:
features = [ 'pretende_fazer_cursos_lit', 'interesse_mba_lit',
             'importante_ter_certificado', 'horas_semanais_estudo',
             'total_modulos', 'modulos_iniciados', 'modulos_finalizados',
             'certificados', 'code_graduacao', 'code_universidade',
             'code_profissao', 'code_organizacao', 'code_como_conheceu_lit'
           ]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_tbs[features]


In [15]:
%%time
#Rodar previsao no modelo para gerar targets
y_pred = RFC.predict(X)
#Popular coluna categoria com os targets previstos pelo modelo
df_to_be_scored['target'] = y_pred

df_to_be_scored.head()

Wall time: 615 ms


id       graduacao          universidade        profissao  \
0  648443203     Bacharelado                 Unesp           Outros   
1  423266690  Especialização               UNICAMP  SEM EXPERIÊNCIA   
2  315097368     Bacharelado               UNICAMP       Engenheiro   
3  911835279    Licenciatura               UNICAMP           Outros   
4  755852400  Especialização  FACULDADE ANHANGUERA       Engenheiro   

        organizacao  pretende_fazer_cursos_lit  interesse_mba_lit  \
0          Borracha                          0                  0   
1  Eletrodomesticos                          1                  1   
2        Oleo e Gas                          0                  0   
3            Estado                          1                  1   
4    Entretenimento                          0                  0   

   importante_ter_certificado  horas_semanais_estudo  \
0                           1                      3   
1                           1                      7   
2                           1                      9   
3                           1                      9   
4                           1                      8   

                   como_conheceu_lit  total_modulos  modulos_iniciados  \
0                         Saint Paul             58                  3   
1         Mídia (revista/jornal/web)             62                 19   
2                             Google             36                  3   
3                             Google            145                 61   
4  Minha empresa - benefício LITpass             33                 33   

   modulos_finalizados  certificados   target  
0                    2             0  perfil6  
1                   17             0  perfil3  
2                    1             0  perfil6  
3                   54             1  perfil1  
4                   29             0  perfil2

### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [16]:
df_to_be_scored.to_csv(r'C:/Users/MARCOSPAULOFerreira/Anaconda2/MARATONA2020/desafio-6-2020/dataset/results.csv', index = False)
#project.save_data(file_name="results.csv", data=df_to_be_scored_4.to_csv(index=False))

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://lit.maratona.dev
